In [1]:
#Importing required libraries

import imblearn
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier


In [2]:
# Copy train dataset into dataframe

train_data_file = '/Users/mackbookair/Desktop/Git/CapStone/CapstoneProject/TestData/allstate-purchase-prediction-challenge-datawargling/train.csv'
train_df = pd.read_csv(train_data_file)
train_df.head()

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,...,C_previous,duration_previous,A,B,C,D,E,F,G,cost
0,10000000,1,0,0,08:35,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,2,633
1,10000000,2,0,0,08:38,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,1,630
2,10000000,3,0,0,08:38,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,1,630
3,10000000,4,0,0,08:39,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,1,630
4,10000000,5,0,0,11:55,IN,10001,2,0,2,...,1.0,2.0,1,0,2,2,1,2,1,630


In [3]:
train_df.describe()

,customer_ID,shopping_pt,record_type,day,location,group_size,homeowner,car_age,risk_factor,age_oldest,...,C_previous,duration_previous,A,B,C,D,E,F,G,cost
count,6.652490e+05,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,424831.000000,665249.000000,...,646538.000000,646538.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000
mean,1.007655e+07,4.219966,0.145824,1.969429,12271.543022,1.234784,0.536229,8.139437,2.563996,44.992403,...,2.444718,6.003774,0.927546,0.454236,2.275265,2.454299,0.445193,1.162768,2.275120,635.785008
std,4.404978e+04,2.394369,0.352930,1.453470,1564.789415,0.461036,0.498686,5.764598,1.111584,17.403440,...,1.034596,4.680793,0.595221,0.497902,0.990798,0.753953,0.496987,0.943594,0.909567,45.993758
min,1.000000e+07,1.000000,0.000000,0.000000,10001.000000,1.000000,0.000000,0.000000,1.000000,18.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,260.000000
25%,1.003852e+07,2.000000,0.000000,1.000000,10936.000000,1.000000,0.000000,3.000000,2.000000,28.000000,...,1.000000,2.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,2.000000,605.000000
50%,1.007640e+07,4.000000,0.000000,2.000000,12027.000000,1.000000,1.000000,7.000000,3.000000,44.000000,...,3.000000,5.000000,1.000000,0.000000,2.000000,3.000000,0.000000,1.000000,2.000000,635.000000
75%,1.011470e+07,6.000000,0.000000,3.000000,13426.000000,1.000000,1.000000,12.000000,4.000000,60.000000,...,3.000000,9.000000,1.000000,1.000000,3.000000,3.000000,1.000000,2.000000,3.000000,665.000000
max,1.015272e+07,13.000000,1.000000,6.000000,16580.000000,4.000000,1.000000,85.000000,4.000000,75.000000,...,4.000000,15.000000,2.000000,1.000000,4.000000,3.000000,1.000000,3.000000,4.000000,922.000000


In [4]:
train_df.columns

Index(['customer_ID', 'shopping_pt', 'record_type', 'day', 'time', 'state',
       'location', 'group_size', 'homeowner', 'car_age', 'car_value',
       'risk_factor', 'age_oldest', 'age_youngest', 'married_couple',
       'C_previous', 'duration_previous', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
       'cost'],
      dtype='object')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665249 entries, 0 to 665248
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_ID        665249 non-null  int64  
 1   shopping_pt        665249 non-null  int64  
 2   record_type        665249 non-null  int64  
 3   day                665249 non-null  int64  
 4   time               665249 non-null  object 
 5   state              665249 non-null  object 
 6   location           665249 non-null  int64  
 7   group_size         665249 non-null  int64  
 8   homeowner          665249 non-null  int64  
 9   car_age            665249 non-null  int64  
 10  car_value          663718 non-null  object 
 11  risk_factor        424831 non-null  float64
 12  age_oldest         665249 non-null  int64  
 13  age_youngest       665249 non-null  int64  
 14  married_couple     665249 non-null  int64  
 15  C_previous         646538 non-null  float64
 16  du

Observations:

1. Total 665,249 records

2. 25 columns

3. Different data types -   
    a. time is identified as object      
    b. state and car_value are other columns which are also identified as objects   
    c. risk_factor, C_previous and duration_previous are identified as float  
    d. all other columns are identified as int  
    
4. Following columns have Null values -
    car_value, 
    risk_factor, 
    C_previous, 
    duration_previous
    
5. column name 'customer_ID' is inconsistent

6. Missing values:     
    a. 240,418 records does not have value for risk_factor, which approximates to 36% of total records  
    b. 1,531 records does not have value for car_value, which approximates to 0.23% of total records  
    c. 18,711 records does not have value for C_previous, which approximates to 2.81% of total records   
    d. 18,711 records does not have value for duration_previous, which approximates to 2.81% of total records

Lot of records does not have a value for risk_factor. 

In [6]:
# Chekcing for unique values in risk_factor
train_df.risk_factor.unique()

array([ 3.,  4., nan,  2.,  1.])

In [7]:
# Checking for number of records for each distinct value
train_df['risk_factor'].value_counts()

3.0    117571
4.0    110754
1.0     99476
2.0     97030
Name: risk_factor, dtype: int64

In [8]:
# Caliculating mean of the column
risk_factor_mean = round(train_df['risk_factor'].mean())
risk_factor_mean

3

In [9]:
# Replacing na values with the mean value 
train_df['risk_factor']=train_df['risk_factor'].fillna(round((train_df['risk_factor'].mean())))
train_df['risk_factor'].value_counts()

3.0    357989
4.0    110754
1.0     99476
2.0     97030
Name: risk_factor, dtype: int64

In [10]:
train_df.risk_factor.unique()

array([3., 4., 2., 1.])

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665249 entries, 0 to 665248
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_ID        665249 non-null  int64  
 1   shopping_pt        665249 non-null  int64  
 2   record_type        665249 non-null  int64  
 3   day                665249 non-null  int64  
 4   time               665249 non-null  object 
 5   state              665249 non-null  object 
 6   location           665249 non-null  int64  
 7   group_size         665249 non-null  int64  
 8   homeowner          665249 non-null  int64  
 9   car_age            665249 non-null  int64  
 10  car_value          663718 non-null  object 
 11  risk_factor        665249 non-null  float64
 12  age_oldest         665249 non-null  int64  
 13  age_youngest       665249 non-null  int64  
 14  married_couple     665249 non-null  int64  
 15  C_previous         646538 non-null  float64
 16  du

In [12]:
# Droping rest of records having nan values
train_df = train_df.dropna()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_ID        645648 non-null  int64  
 1   shopping_pt        645648 non-null  int64  
 2   record_type        645648 non-null  int64  
 3   day                645648 non-null  int64  
 4   time               645648 non-null  object 
 5   state              645648 non-null  object 
 6   location           645648 non-null  int64  
 7   group_size         645648 non-null  int64  
 8   homeowner          645648 non-null  int64  
 9   car_age            645648 non-null  int64  
 10  car_value          645648 non-null  object 
 11  risk_factor        645648 non-null  float64
 12  age_oldest         645648 non-null  int64  
 13  age_youngest       645648 non-null  int64  
 14  married_couple     645648 non-null  int64  
 15  C_previous         645648 non-null  float64
 16  du

We have 645,648 out of 665,249 total records, we droped around 3% of records which have nan value.

In [13]:
#Renaming column names - converting uppercase names to lower case
train_df.rename(columns={'customer_ID':'customer_id','C_previous':'c_previous','A':'a','B':'b','C':'c','D':'d','E':'e','F':'f','G':'g'},inplace = True)

train_df.columns

Index(['customer_id', 'shopping_pt', 'record_type', 'day', 'time', 'state',
       'location', 'group_size', 'homeowner', 'car_age', 'car_value',
       'risk_factor', 'age_oldest', 'age_youngest', 'married_couple',
       'c_previous', 'duration_previous', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
       'cost'],
      dtype='object')

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_id        645648 non-null  int64  
 1   shopping_pt        645648 non-null  int64  
 2   record_type        645648 non-null  int64  
 3   day                645648 non-null  int64  
 4   time               645648 non-null  object 
 5   state              645648 non-null  object 
 6   location           645648 non-null  int64  
 7   group_size         645648 non-null  int64  
 8   homeowner          645648 non-null  int64  
 9   car_age            645648 non-null  int64  
 10  car_value          645648 non-null  object 
 11  risk_factor        645648 non-null  float64
 12  age_oldest         645648 non-null  int64  
 13  age_youngest       645648 non-null  int64  
 14  married_couple     645648 non-null  int64  
 15  c_previous         645648 non-null  float64
 16  du

In [15]:
train_df.time.unique()

array(['08:35', '08:38', '08:39', ..., '00:13', '00:16', '00:40'],
      dtype=object)

In [16]:
# dropping time column
train_df = train_df.drop('time',axis=1)
train_df.columns

Index(['customer_id', 'shopping_pt', 'record_type', 'day', 'state', 'location',
       'group_size', 'homeowner', 'car_age', 'car_value', 'risk_factor',
       'age_oldest', 'age_youngest', 'married_couple', 'c_previous',
       'duration_previous', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'cost'],
      dtype='object')

In [17]:
train_df.customer_id.unique()

array([10000000, 10000005, 10000007, ..., 10152721, 10152723, 10152724])

In [18]:
# dropping customer_id column
train_df = train_df.drop('customer_id',axis=1)
train_df.columns

Index(['shopping_pt', 'record_type', 'day', 'state', 'location', 'group_size',
       'homeowner', 'car_age', 'car_value', 'risk_factor', 'age_oldest',
       'age_youngest', 'married_couple', 'c_previous', 'duration_previous',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'cost'],
      dtype='object')

In [19]:
train_df.day.unique()

array([0, 3, 4, 2, 1, 5, 6])

In [20]:
train_df['day'].value_counts()

0    136441
1    132807
2    129654
4    119937
3    117781
5      8088
6       940
Name: day, dtype: int64

In [21]:
train_df['day'][train_df.record_type==0].value_counts()

0    118009
1    113738
2    110982
3     99980
4     99273
5      6672
6       821
Name: day, dtype: int64

In [22]:
# dropping day column
train_df = train_df.drop('day',axis=1)
train_df.columns

Index(['shopping_pt', 'record_type', 'state', 'location', 'group_size',
       'homeowner', 'car_age', 'car_value', 'risk_factor', 'age_oldest',
       'age_youngest', 'married_couple', 'c_previous', 'duration_previous',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'cost'],
      dtype='object')

In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   shopping_pt        645648 non-null  int64  
 1   record_type        645648 non-null  int64  
 2   state              645648 non-null  object 
 3   location           645648 non-null  int64  
 4   group_size         645648 non-null  int64  
 5   homeowner          645648 non-null  int64  
 6   car_age            645648 non-null  int64  
 7   car_value          645648 non-null  object 
 8   risk_factor        645648 non-null  float64
 9   age_oldest         645648 non-null  int64  
 10  age_youngest       645648 non-null  int64  
 11  married_couple     645648 non-null  int64  
 12  c_previous         645648 non-null  float64
 13  duration_previous  645648 non-null  float64
 14  a                  645648 non-null  int64  
 15  b                  645648 non-null  int64  
 16  c 

Inspecting columns with type as float64

In [24]:
train_df.risk_factor.unique()

array([3., 4., 2., 1.])

In [25]:
# Convert risk_factor column to int type
train_df['risk_factor']=train_df['risk_factor'].astype('int')
train_df.risk_factor.unique()

array([3, 4, 2, 1])

In [26]:
train_df.c_previous.unique()

array([1., 3., 2., 4.])

In [27]:
# Convert c_previous column to int type
train_df['c_previous']=train_df['c_previous'].astype('int')
train_df.c_previous.unique()

array([1, 3, 2, 4])

In [28]:
 train_df.duration_previous.unique()

array([ 2., 13.,  4.,  3.,  9.,  0.,  1.,  8., 14.,  5., 15.,  6.,  7.,
       11., 10., 12.])

In [29]:
# Convert duration_previous column to int type
train_df['duration_previous']=train_df['duration_previous'].astype('int')
train_df.duration_previous.unique()

array([ 2, 13,  4,  3,  9,  0,  1,  8, 14,  5, 15,  6,  7, 11, 10, 12])

In [30]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   shopping_pt        645648 non-null  int64 
 1   record_type        645648 non-null  int64 
 2   state              645648 non-null  object
 3   location           645648 non-null  int64 
 4   group_size         645648 non-null  int64 
 5   homeowner          645648 non-null  int64 
 6   car_age            645648 non-null  int64 
 7   car_value          645648 non-null  object
 8   risk_factor        645648 non-null  int64 
 9   age_oldest         645648 non-null  int64 
 10  age_youngest       645648 non-null  int64 
 11  married_couple     645648 non-null  int64 
 12  c_previous         645648 non-null  int64 
 13  duration_previous  645648 non-null  int64 
 14  a                  645648 non-null  int64 
 15  b                  645648 non-null  int64 
 16  c                  6

In [31]:
train_df.head()

,shopping_pt,record_type,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,...,c_previous,duration_previous,a,b,c,d,e,f,g,cost
0,1,0,IN,10001,2,0,2,g,3,46,...,1,2,1,0,2,2,1,2,2,633
1,2,0,IN,10001,2,0,2,g,3,46,...,1,2,1,0,2,2,1,2,1,630
2,3,0,IN,10001,2,0,2,g,3,46,...,1,2,1,0,2,2,1,2,1,630
3,4,0,IN,10001,2,0,2,g,3,46,...,1,2,1,0,2,2,1,2,1,630
4,5,0,IN,10001,2,0,2,g,3,46,...,1,2,1,0,2,2,1,2,1,630


In [33]:
train_df.state.unique()

array(['IN', 'NY', 'PA', 'WV', 'MO', 'OH', 'OK', 'FL', 'OR', 'WA', 'KS',
       'NV', 'ID', 'CO', 'CT', 'AL', 'AR', 'NM', 'MS', 'MD', 'RI', 'ME',
       'TN', 'UT', 'WI', 'MT', 'KY', 'WY', 'NE', 'ND', 'DE', 'GA', 'NH',
       'IA', 'DC', 'SD'], dtype=object)

In [34]:
train_df['state'].value_counts()

FL    104988
NY     89589
PA     59049
OH     43417
MD     27957
WA     24537
IN     23891
CO     23260
AL     22972
CT     18847
TN     17537
KY     15589
NV     15092
MO     14759
OR     14207
UT     14167
OK     13526
MS     10258
AR     10223
WI      9954
GA      8484
NH      7312
NM      6676
ME      6370
ID      5873
RI      5711
KS      5490
WV      4989
IA      4436
DE      3996
DC      3271
MT      3110
NE      3087
ND      1454
WY      1133
SD       437
Name: state, dtype: int64

In [35]:
train_df['state'][train_df.record_type==1].value_counts()

FL    14599
NY    13574
PA     8900
OH     6641
MD     4218
IN     3639
AL     3578
WA     3561
CO     3516
CT     2813
TN     2635
KY     2348
MO     2193
OR     2138
NV     2119
UT     2069
OK     2018
AR     1561
MS     1557
WI     1544
GA     1254
NH     1091
ME     1002
NM      999
ID      885
RI      873
KS      858
WV      779
IA      686
DE      595
DC      502
NE      490
MT      461
ND      240
WY      166
SD       71
Name: state, dtype: int64

In [36]:
train_df.location.unique()

array([10001, 10006, 10008, ..., 16579, 16562, 16580])

In [37]:
train_df['location'].value_counts()

10083    1027
10213     974
11517     864
10348     860
10030     782
         ... 
16299       1
16166       1
16026       1
16140       1
16405       1
Name: location, Length: 6244, dtype: int64

In [38]:
train_df['group_size'].value_counts()

1    503170
2    133112
3      8686
4       680
Name: group_size, dtype: int64

In [39]:
train_df['group_size'][train_df.record_type==1].value_counts()

1    74924
2    19865
3     1274
4      110
Name: group_size, dtype: int64

In [40]:
train_df['homeowner'].value_counts()

1    348357
0    297291
Name: homeowner, dtype: int64

In [41]:
train_df['homeowner'][train_df.record_type==1].value_counts()

1    52620
0    43553
Name: homeowner, dtype: int64

In [42]:
train_df['car_age'].value_counts()

1     69635
2     49355
7     45681
6     43408
8     43217
      ...  
65        5
51        5
60        4
85        4
54        1
Name: car_age, Length: 67, dtype: int64

In [43]:
train_df['car_age'][train_df.record_type==1].value_counts()

1     10290
2      7292
7      6772
8      6424
6      6399
      ...  
55        1
57        1
58        1
65        1
60        1
Name: car_age, Length: 65, dtype: int64

In [44]:
train_df['car_value'].value_counts()

e    213495
f    172456
d    109861
g     95497
h     28224
c     20179
i      3520
b      1342
a      1074
Name: car_value, dtype: int64

In [45]:
train_df['car_value'][train_df.record_type==1].value_counts()

e    31884
f    25725
d    16252
g    14283
h     4126
c     3027
i      500
b      206
a      170
Name: car_value, dtype: int64

In [46]:
train_df['age_oldest'].value_counts()

75    44133
24    20875
23    20751
25    20617
22    18930
26    18121
27    15494
28    14751
21    13686
29    12771
30    12371
31    11636
32    11279
33    11079
43    11039
55    10842
42    10747
52    10686
34    10658
53    10490
46    10458
35    10385
56    10380
49    10308
54    10283
48    10268
58    10211
51    10130
50     9983
59     9908
45     9860
44     9839
41     9815
20     9700
36     9656
57     9615
47     9589
37     9532
66     9518
65     9455
63     9444
38     9440
60     9417
40     9321
61     9257
39     9192
62     9149
64     9076
70     8092
69     7458
67     7391
68     6819
71     6738
19     6210
72     6055
73     5724
74     5551
18     1465
Name: age_oldest, dtype: int64

In [47]:
train_df['age_youngest'].value_counts()

75    33482
23    23386
25    22779
24    22765
22    21464
26    19886
27    17154
21    16677
28    16383
29    14013
30    13268
20    12903
31    12625
32    12137
33    11663
34    11139
35    10821
55     9992
43     9858
42     9826
52     9715
19     9678
36     9624
48     9507
37     9495
46     9469
58     9454
53     9452
51     9382
49     9371
56     9349
38     9316
50     9263
54     9251
41     9223
40     9084
45     9039
44     8981
65     8936
57     8894
39     8834
60     8686
47     8674
59     8666
66     8661
62     8539
61     8490
63     8415
64     8353
70     7142
69     6870
68     6295
67     6075
71     6021
72     5148
73     5069
74     4931
18     4835
16     4380
17     2860
Name: age_youngest, dtype: int64

In [48]:
train_df['married_couple'].value_counts()

0    509473
1    136175
Name: married_couple, dtype: int64

In [49]:
train_df['c_previous'].value_counts()

3    270882
1    171605
2    109025
4     94136
Name: c_previous, dtype: int64

In [50]:
train_df['duration_previous'].value_counts()

1     81270
15    79825
2     79388
3     70682
4     57417
5     49332
6     45356
7     37754
8     30734
9     26235
0     24874
10    18092
11    12715
12    11282
13    10955
14     9737
Name: duration_previous, dtype: int64

In [51]:
train_df['a'].value_counts()

1    416650
0    135744
2     93254
Name: a, dtype: int64

In [52]:
train_df['b'].value_counts()

0    349542
1    296106
Name: b, dtype: int64

In [53]:
train_df['c'].value_counts()

3    266103
1    193609
2    129597
4     56339
Name: c, dtype: int64

In [54]:
train_df['d'].value_counts()

3    401183
2    145026
1     99439
Name: d, dtype: int64

In [55]:
train_df['e'].value_counts()

0    354557
1    291091
Name: e, dtype: int64

In [56]:
train_df['f'].value_counts()

2    249890
0    206957
1    155474
3     33327
Name: f, dtype: int64

In [57]:
train_df['g'].value_counts()

2    251112
3    189315
1    139280
4     65941
Name: g, dtype: int64

In [58]:
train_df['cost'].value_counts()

633    5901
637    5869
626    5824
638    5808
640    5801
       ... 
418       1
416       1
851       1
408       1
860       1
Name: cost, Length: 524, dtype: int64

In [59]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   shopping_pt        645648 non-null  int64 
 1   record_type        645648 non-null  int64 
 2   state              645648 non-null  object
 3   location           645648 non-null  int64 
 4   group_size         645648 non-null  int64 
 5   homeowner          645648 non-null  int64 
 6   car_age            645648 non-null  int64 
 7   car_value          645648 non-null  object
 8   risk_factor        645648 non-null  int64 
 9   age_oldest         645648 non-null  int64 
 10  age_youngest       645648 non-null  int64 
 11  married_couple     645648 non-null  int64 
 12  c_previous         645648 non-null  int64 
 13  duration_previous  645648 non-null  int64 
 14  a                  645648 non-null  int64 
 15  b                  645648 non-null  int64 
 16  c                  6

Converting 'state','location','group_size','homeowner','car_value','risk_factor','married_couple','c_previous','a','b','c','d','e','f' and 'g' columns to category type

In [60]:
categorical_cols = ['state','location','group_size','homeowner','car_value','risk_factor','married_couple',
                    'c_previous','a','b','c','d','e','f','g'] 

In [61]:
for col in categorical_cols:
  train_df[col] = train_df[col].astype('category')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   shopping_pt        645648 non-null  int64   
 1   record_type        645648 non-null  int64   
 2   state              645648 non-null  category
 3   location           645648 non-null  category
 4   group_size         645648 non-null  category
 5   homeowner          645648 non-null  category
 6   car_age            645648 non-null  int64   
 7   car_value          645648 non-null  category
 8   risk_factor        645648 non-null  category
 9   age_oldest         645648 non-null  int64   
 10  age_youngest       645648 non-null  int64   
 11  married_couple     645648 non-null  category
 12  c_previous         645648 non-null  category
 13  duration_previous  645648 non-null  int64   
 14  a                  645648 non-null  category
 15  b                  645648 non-null

In [62]:
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
train_df[categorical_cols] = train_df[categorical_cols].apply(lambda col: le.fit_transform(col))
train_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 645648 entries, 0 to 665248
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   shopping_pt        645648 non-null  int64
 1   record_type        645648 non-null  int64
 2   state              645648 non-null  int64
 3   location           645648 non-null  int64
 4   group_size         645648 non-null  int64
 5   homeowner          645648 non-null  int64
 6   car_age            645648 non-null  int64
 7   car_value          645648 non-null  int64
 8   risk_factor        645648 non-null  int64
 9   age_oldest         645648 non-null  int64
 10  age_youngest       645648 non-null  int64
 11  married_couple     645648 non-null  int64
 12  c_previous         645648 non-null  int64
 13  duration_previous  645648 non-null  int64
 14  a                  645648 non-null  int64
 15  b                  645648 non-null  int64
 16  c                  645648 non-null  in

In [63]:
train_df['record_type'].value_counts()

0    549475
1     96173
Name: record_type, dtype: int64

There are 549,475 records with 0 record_type and only 96,173 records with 1. Which is about 14% of total records 645,648 records. 

Any model trained on this data set would have over sampling records for record_type as 1)

In [64]:
df_s = train_df
df_sample = df_s.sample(frac=0.1, replace=True)

In [65]:
df_sample.head()

,shopping_pt,record_type,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,...,c_previous,duration_previous,a,b,c,d,e,f,g,cost
358849,6,0,24,4842,0,1,1,6,2,26,...,2,6,1,0,2,2,0,2,3,655
601748,4,0,6,898,0,0,9,7,0,34,...,2,4,1,0,0,1,0,2,2,652
532726,6,0,30,2429,0,0,11,4,2,38,...,1,3,1,0,1,1,1,2,1,660
498727,1,0,6,2657,0,0,14,4,2,31,...,1,2,1,1,0,0,1,2,3,684
594418,2,0,23,509,0,0,10,3,1,27,...,2,4,1,1,2,2,0,0,0,624


In [66]:
df_sample.info

<bound method DataFrame.info of         shopping_pt  record_type  state  location  group_size  homeowner  \
358849            6            0     24      4842           0          1   
601748            4            0      6       898           0          0   
532726            6            0     30      2429           0          0   
498727            1            0      6      2657           0          0   
594418            2            0     23       509           0          0   
...             ...          ...    ...       ...         ...        ...   
588519            2            0     24      1484           2          1   
370747            6            0     23       361           0          1   
483541            3            0      2       796           0          1   
442561            6            0     32      4737           0          0   
197689            9            1     23      3306           0          0   

        car_age  car_value  risk_factor  age_oldest  ..

In [67]:
print(imblearn.__version__)

0.7.0


In [68]:
train_df = df_sample

In [69]:
X_df = train_df.drop('record_type',axis=1)

In [70]:
y_recordtype = train_df['record_type']
y_recordtype

358849    0
601748    0
532726    0
498727    0
594418    0
         ..
588519    0
370747    0
483541    0
442561    0
197689    1
Name: record_type, Length: 64565, dtype: int64

In [71]:
# Split the data into a training and test set.
Xlr, Xtestlr, ylr, ytestlr = train_test_split(X_df, 
                                              y_recordtype, test_size=0.1,random_state=42)

In [72]:
Xlr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58108 entries, 598565 to 585542
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   shopping_pt        58108 non-null  int64
 1   state              58108 non-null  int64
 2   location           58108 non-null  int64
 3   group_size         58108 non-null  int64
 4   homeowner          58108 non-null  int64
 5   car_age            58108 non-null  int64
 6   car_value          58108 non-null  int64
 7   risk_factor        58108 non-null  int64
 8   age_oldest         58108 non-null  int64
 9   age_youngest       58108 non-null  int64
 10  married_couple     58108 non-null  int64
 11  c_previous         58108 non-null  int64
 12  duration_previous  58108 non-null  int64
 13  a                  58108 non-null  int64
 14  b                  58108 non-null  int64
 15  c                  58108 non-null  int64
 16  d                  58108 non-null  int64
 17  e     

In [73]:
ylr.count()

58108

In [74]:
ylr.value_counts()

0    49475
1     8633
Name: record_type, dtype: int64

In [75]:
oversample = SMOTE()

In [76]:
X, y = oversample.fit_resample(Xlr, ylr)

In [77]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98950 entries, 0 to 98949
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   shopping_pt        98950 non-null  int64
 1   state              98950 non-null  int64
 2   location           98950 non-null  int64
 3   group_size         98950 non-null  int64
 4   homeowner          98950 non-null  int64
 5   car_age            98950 non-null  int64
 6   car_value          98950 non-null  int64
 7   risk_factor        98950 non-null  int64
 8   age_oldest         98950 non-null  int64
 9   age_youngest       98950 non-null  int64
 10  married_couple     98950 non-null  int64
 11  c_previous         98950 non-null  int64
 12  duration_previous  98950 non-null  int64
 13  a                  98950 non-null  int64
 14  b                  98950 non-null  int64
 15  c                  98950 non-null  int64
 16  d                  98950 non-null  int64
 17  e           

In [78]:
y.count()

98950

In [79]:
y.value_counts()

1    49475
0    49475
Name: record_type, dtype: int64

After applying SMOTE, number of successful records 1 increased from 57671 to 329717. Now its 50%

In [80]:
# Applyting LogisticRegression

clf = LogisticRegression()
# Fit the model on the trainng data.
clf.fit(X, y)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(Xtestlr), ytestlr))

0.7518971658665015


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [81]:
print(ytestlr.value_counts())

0    5525
1     932
Name: record_type, dtype: int64


In [82]:
print(confusion_matrix(ytestlr, clf.predict(Xtestlr)))

[[4168 1357]
 [ 245  687]]


In [83]:
print(classification_report(ytestlr, clf.predict(Xtestlr)))

              precision    recall  f1-score   support

           0       0.94      0.75      0.84      5525
           1       0.34      0.74      0.46       932

    accuracy                           0.75      6457
   macro avg       0.64      0.75      0.65      6457
weighted avg       0.86      0.75      0.78      6457



In [84]:
# Applying KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=6)

In [85]:
knn.fit(X,y)
y_pred_knn = knn.predict(Xtestlr)

In [86]:
print(confusion_matrix(ytestlr, y_pred_knn))

[[3726 1799]
 [ 583  349]]


In [87]:
print(classification_report(ytestlr, y_pred_knn))

              precision    recall  f1-score   support

           0       0.86      0.67      0.76      5525
           1       0.16      0.37      0.23       932

    accuracy                           0.63      6457
   macro avg       0.51      0.52      0.49      6457
weighted avg       0.76      0.63      0.68      6457



Performing hyperprameter tuning, to check with different n_neighbors values

In [88]:

for k in range(1,5):
    print(" Iterating with n_neighbors ", k)
    knn_h = KNeighborsClassifier(n_neighbors=k)
    knn_h.fit(X,y)
    y_pred_knn_h = knn_h.predict(Xtestlr)
    print(confusion_matrix(ytestlr, y_pred_knn_h))
    print(classification_report(ytestlr, y_pred_knn_h))

 Iterating with n_neighbors  1
[[4364 1161]
 [ 671  261]]
              precision    recall  f1-score   support

           0       0.87      0.79      0.83      5525
           1       0.18      0.28      0.22       932

    accuracy                           0.72      6457
   macro avg       0.53      0.53      0.52      6457
weighted avg       0.77      0.72      0.74      6457

 Iterating with n_neighbors  2
[[4666  859]
 [ 735  197]]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85      5525
           1       0.19      0.21      0.20       932

    accuracy                           0.75      6457
   macro avg       0.53      0.53      0.53      6457
weighted avg       0.77      0.75      0.76      6457

 Iterating with n_neighbors  3
[[3780 1745]
 [ 589  343]]
              precision    recall  f1-score   support

           0       0.87      0.68      0.76      5525
           1       0.16      0.37      0.23       932

    accu

In [89]:
# Applying Random Forest

from sklearn.ensemble import RandomForestClassifier

In [90]:
# define the model
rfc = RandomForestClassifier()
# fit the model on the whole dataset
rfc.fit(X, y)

RandomForestClassifier()

In [91]:
yhat = rfc.predict(Xtestlr)

In [92]:
yhat

array([0, 0, 0, ..., 0, 1, 0])

In [93]:
print(confusion_matrix(ytestlr, yhat))

[[4803  722]
 [ 480  452]]


In [94]:
print(classification_report(ytestlr, yhat))

              precision    recall  f1-score   support

           0       0.91      0.87      0.89      5525
           1       0.39      0.48      0.43       932

    accuracy                           0.81      6457
   macro avg       0.65      0.68      0.66      6457
weighted avg       0.83      0.81      0.82      6457



In [96]:
from sklearn.svm import OneClassSVM
from sklearn import svm

In [97]:
#svm_clf = OneClassSVM(kernel='linear',gamma='auto',class_weight='').fit(X)

#svm.SVC(kernel='linear', class_weight={1: 10})

svm_clf = svm.SVC(kernel='linear', cache_size=7000,C=1.0)
#
#svm_clf =svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

In [101]:
Xlr.info

<bound method DataFrame.info of         shopping_pt  state  location  group_size  homeowner  car_age  \
598565           11      6       686           0          1        1   
242179            1     23       388           1          1        1   
94778             8      2      1014           0          0        1   
414561            5      2      1302           0          1        7   
655056            1     12       689           0          0       10   
...             ...    ...       ...         ...        ...      ...   
230169            2     13      2324           0          1        1   
467555            5     27      1029           1          1       16   
319089            8     24      1089           1          1        1   
299912            1     10      4918           0          0       12   
585542            4      6       253           0          0        9   

        car_value  risk_factor  age_oldest  age_youngest  ...  c_previous  \
598565          6         

In [103]:
ylr.value_counts()

0    49475
1     8633
Name: record_type, dtype: int64

In [107]:
df_sample_red = df_s.sample(frac=0.05, replace=True)

In [108]:
df_sample_red.info

<bound method DataFrame.info of         shopping_pt  record_type  state  location  group_size  homeowner  \
582501            6            0     10      2726           0          1   
215736            9            1     27       983           0          0   
178385            3            0     23      4549           0          0   
19055             1            0     32       356           0          1   
585450            3            1      6       932           0          1   
...             ...          ...    ...       ...         ...        ...   
140579            2            0     22      1983           0          1   
284663            3            0     27      2249           1          1   
577346            4            1      6      1807           0          1   
51386             3            0      0      1471           0          1   
292211            4            0      6      3889           0          1   

        car_age  car_value  risk_factor  age_oldest  ..

In [109]:
X_df_red = df_sample_red.drop('record_type',axis=1)
X_df_red

,shopping_pt,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,...,c_previous,duration_previous,a,b,c,d,e,f,g,cost
582501,6,10,2726,0,1,7,6,3,24,24,...,0,1,1,0,0,0,0,2,0,674
215736,9,27,983,0,0,1,7,2,30,30,...,2,13,1,0,2,2,0,1,1,692
178385,3,23,4549,0,0,3,4,1,38,38,...,2,3,1,0,2,2,1,0,2,658
19055,1,32,356,0,1,1,5,2,38,38,...,2,3,1,0,1,2,0,2,3,667
585450,3,6,932,0,1,7,3,0,26,26,...,0,11,1,1,0,0,0,2,2,613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140579,2,22,1983,0,1,5,8,3,75,75,...,2,3,1,0,2,2,0,1,0,735
284663,3,27,2249,1,1,8,7,2,75,56,...,2,2,1,1,2,2,1,2,0,645
577346,4,6,1807,0,1,18,5,1,59,59,...,2,4,1,0,2,2,0,2,2,625
51386,3,0,1471,0,1,15,5,2,23,23,...,2,7,0,0,0,1,0,0,1,574


In [112]:
y_recordtype_red = df_sample_red['record_type']
y_recordtype_red

582501    0
215736    1
178385    0
19055     0
585450    1
         ..
140579    0
284663    0
577346    1
51386     0
292211    0
Name: record_type, Length: 32282, dtype: int64

In [113]:
# Split the data into a training and test set.
Xlr_red, Xtestlr_red, ylr_red, ytestlr_red = train_test_split(X_df_red, 
                                              y_recordtype_red, test_size=0.1,random_state=42)

In [114]:
Xlr_red.info

<bound method DataFrame.info of         shopping_pt  state  location  group_size  homeowner  car_age  \
476318            4     31        84           0          0        3   
616684            2     23      3948           0          0        0   
181726            5      2      3978           0          0        6   
537925            5      6      3295           0          1        7   
353160            4     27       138           1          0        4   
...             ...    ...       ...         ...        ...      ...   
134758            2      6      1039           0          0        7   
522117            5      6      3974           1          1        2   
141607            7     15      1683           0          1        3   
340467            5      6      1548           0          1        4   
640552            6     23      2036           0          0        6   

        car_value  risk_factor  age_oldest  age_youngest  ...  c_previous  \
476318          3         

In [118]:
ylr_red.count

<bound method Series.count of 476318    0
616684    0
181726    0
537925    0
353160    0
         ..
134758    0
522117    0
141607    0
340467    0
640552    1
Name: record_type, Length: 29053, dtype: int64>

In [119]:
ylr_red.value_counts()

0    24673
1     4380
Name: record_type, dtype: int64

In [120]:
svm_clf.fit(Xlr_red, ylr_red)

SVC(cache_size=7000, kernel='linear')

In [121]:
y_svm = svm_clf.predict(Xtestlr_red)

In [123]:
print(confusion_matrix(ytestlr_red, y_svm))

[[2582  155]
 [ 350  142]]


In [124]:
print(classification_report(ytestlr_red, y_svm))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      2737
           1       0.48      0.29      0.36       492

    accuracy                           0.84      3229
   macro avg       0.68      0.62      0.64      3229
weighted avg       0.82      0.84      0.83      3229



In [125]:
svm_clf_rbf = svm.SVC(kernel='rbf', cache_size=7000,C=1.0)

In [126]:
svm_clf_rbf.fit(Xlr_red, ylr_red)

SVC(cache_size=7000)

In [127]:
y_svm_rbf = svm_clf.predict(Xtestlr_red)

In [128]:
print(confusion_matrix(ytestlr_red, y_svm_rbf))

[[2582  155]
 [ 350  142]]


In [129]:
print(classification_report(ytestlr_red, y_svm_rbf))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      2737
           1       0.48      0.29      0.36       492

    accuracy                           0.84      3229
   macro avg       0.68      0.62      0.64      3229
weighted avg       0.82      0.84      0.83      3229



In [130]:
svm_clf_rbf_C2 = svm.SVC(kernel='rbf', cache_size=7000,C=2.0)

In [131]:
svm_clf_rbf_C2.fit(Xlr_red, ylr_red)

SVC(C=2.0, cache_size=7000)

In [132]:
y_svm_rbf_C2 = svm_clf_rbf_C2.predict(Xtestlr_red)

In [133]:
print(confusion_matrix(ytestlr_red, y_svm_rbf_C2))

[[2737    0]
 [ 492    0]]


In [134]:
print(classification_report(ytestlr_red, y_svm_rbf_C2))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92      2737
           1       0.00      0.00      0.00       492

    accuracy                           0.85      3229
   macro avg       0.42      0.50      0.46      3229
weighted avg       0.72      0.85      0.78      3229



/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [144]:
# train with only 0 record types

svm_clf_rbf_0rec = OneClassSVM(gamma='scale', nu=0.01)

In [145]:
Xlr_red_0 = Xlr_red[ylr_red==0]
#svm_clf_rbf.fit(Xlr_red)

#trainX = trainX[trainy==0]

In [146]:
svm_clf_rbf_0rec.fit(Xlr_red_0)

OneClassSVM(nu=0.01)

In [147]:
ytestlr_red_0=ytestlr_red

In [149]:
from sklearn.metrics import f1_score
# detect outliers in the test set
yhat_svm_clf_rbf_0rec = svm_clf_rbf_0rec.predict(Xtestlr_red)
# mark inliers 1, outliers -1
ytestlr_red_0[ytestlr_red_0 == 0] = -1
ytestlr_red_0[ytestlr_red_0 == 1] = 1
# calculate score
score = f1_score(ytestlr_red_0, yhat_svm_clf_rbf_0rec, pos_label=-1)
print('F1 Score: %.3f' % score)

F1 Score: 0.019


In [159]:
print(confusion_matrix(ytestlr_red_0, yhat_svm_clf_rbf_0rec))

[[  27 2710]
 [   6  486]]


In [158]:
print(classification_report(ytestlr_red_0, yhat_svm_clf_rbf_0rec))

              precision    recall  f1-score   support

          -1       0.82      0.01      0.02      2737
           1       0.15      0.99      0.26       492

    accuracy                           0.16      3229
   macro avg       0.49      0.50      0.14      3229
weighted avg       0.72      0.16      0.06      3229



In [153]:
svm_clf_linear_C2_OCS = OneClassSVM(kernel='linear', cache_size=7000)

In [154]:
svm_clf_linear_C2_OCS.fit(Xlr_red_0)

OneClassSVM(cache_size=7000, kernel='linear')

In [155]:
yhat_svm_clf_linear_0rec=svm_clf_linear_C2_OCS.predict(Xtestlr_red)

In [156]:
score_l = f1_score(ytestlr_red_0, yhat_svm_clf_linear_0rec, pos_label=-1)
print('F1 Score: %.3f' % score_l)

F1 Score: 0.630


In [157]:
print(classification_report(ytestlr_red_0, yhat_svm_clf_linear_0rec))

              precision    recall  f1-score   support

          -1       0.85      0.50      0.63      2737
           1       0.16      0.51      0.24       492

    accuracy                           0.50      3229
   macro avg       0.50      0.51      0.43      3229
weighted avg       0.74      0.50      0.57      3229



In [160]:
print(confusion_matrix(ytestlr_red_0, yhat_svm_clf_linear_0rec))

[[1370 1367]
 [ 240  252]]
